<a href="https://colab.research.google.com/github/zhannatoleubek-png/special-okx-chainsaw/blob/main/%D0%A0%D0%B0%D0%B7%D0%B2%D0%BE%D1%80%D0%BE%D1%82_%D0%A7%D1%8F%D1%82%D0%B8%D0%BA_upd%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab-ready script: OKX scanner -> Telegram signals
# Требует: ccxt, pandas, numpy, requests
# Установка (в Colab):
!pip install --quiet ccxt pandas numpy requests

# ----------------------------------------------------------------------
# CONFIG — заполните перед запуском
CONFIG = {
    # Telegram
    "telegram_bot_token": "8111633898:AAEH_ssxu0KiyTNHbZ2Ig04ilSTCRGCV6b8",
    # chat_id канала/чата. Если это канал, обычно выглядит как -1001234567890
    "telegram_chat_id": "-4851261476",

    # Список символов, если пустой — будет динамически выбран (включая 'USDT')
    "symbols": [],

    # Сканировать только perpetual / USDT-торговые пары (логика фильтра в коде)
    "market_filter": {"quote": "USDT", "type_contains": ["SWAP", "PERPETUAL", "PERP"]},

    # Частота сканирования в секундах
    "scan_interval": 10 * 60,  # 10 минут

    # Параметры сигналов
    "delta_window_trades": 200,  # сколько последних трейдов смотреть
    "delta_ratio_threshold": 0.6,
    "vwap_window_minutes": 60,   # для интрадей VWAP
    "atr_period": 14,
    "pivot_anchor_tf": "1h",     # TF для расчёта pivot (можно '1h' или '1d')
    "near_level_atr_mult": 0.5,  # buffer around pivot = ATR * this
    "min_confidence_score": 6.0, # порог для отправки сигнала (подстроить при тестах)
    # фильтр минимальной ликвидности: минимальный торговый объём за 24ч (USDT)
    "min_base_volume_24h": 100000
}
# ----------------------------------------------------------------------

import time
import math
import datetime as dt
import traceback
import requests
import pandas as pd
import numpy as np
import ccxt

# ---------------- helpers ----------------
def send_telegram_message(bot_token: str, chat_id: str, text: str, parse_mode: str = "HTML"):
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {"chat_id": chat_id, "text": text, "parse_mode": parse_mode, "disable_web_page_preview": True}
    try:
        r = requests.post(url, json=payload, timeout=10)
        return r.json()
    except Exception as e:
        print("Telegram send error:", e)
        return None

def now_ts():
    return dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")

# ---------------- market helpers ----------------
exchange = ccxt.okx({
    "enableRateLimit": True,
    # Публичные запросы — не указываем ключи
})

def safe_fetch_ohlcv(symbol, timeframe='1h', limit=200):
    try:
        return exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    except Exception as e:
        # пробуем с подключением "swap" суффиксом или другим форматом
        print(f"fetch_ohlcv error for {symbol} {timeframe}: {e}")
        return None

def compute_pivots_from_ohlcv(ohlcv):
    # ohlcv as list of [ts, open, high, low, close, vol]
    # классические пивоты по последней завершенной свечке
    if not ohlcv or len(ohlcv) < 2:
        return None
    prev = ohlcv[-2]  # предпоследняя (т.е. завершенная)
    prev_high = prev[2]
    prev_low = prev[3]
    prev_close = prev[4]
    P = (prev_high + prev_low + prev_close) / 3.0
    R1 = 2*P - prev_low
    S1 = 2*P - prev_high
    R2 = P + (prev_high - prev_low)
    S2 = P - (prev_high - prev_low)
    R3 = prev_high + 2*(P - prev_low)
    S3 = prev_low - 2*(prev_high - P)
    # Camarilla (simplified)
    rng = prev_high - prev_low
    CR = {
        "R1": prev_close + rng * (1.1/12),
        "R2": prev_close + rng * (1.1/6),
        "R3": prev_close + rng * (1.1/4),
        "R4": prev_close + rng * (1.1/2),
        "S1": prev_close - rng * (1.1/12),
        "S2": prev_close - rng * (1.1/6),
        "S3": prev_close - rng * (1.1/4),
        "S4": prev_close - rng * (1.1/2),
    }
    return {"classic": {"P": P, "R1": R1, "S1": S1, "R2": R2, "S2": S2, "R3": R3, "S3": S3}, "camarilla": CR}

def compute_vwap_from_ohlcv(ohlcv):
    # VWAP over ohlcv rows (use typical price * vol)
    df = pd.DataFrame(ohlcv, columns=['ts','open','high','low','close','vol'])
    df['typ'] = (df['high'] + df['low'] + df['close'])/3.0
    df['pv'] = df['typ'] * df['vol']
    cum_pv = df['pv'].sum()
    cum_v = df['vol'].sum()
    vwap = (cum_pv / cum_v) if cum_v > 0 else None
    return vwap

def compute_atr(ohlcv, period=14):
    df = pd.DataFrame(ohlcv, columns=['ts','open','high','low','close','vol'])
    df['prev_close'] = df['close'].shift(1)
    df['tr'] = df[['high','low','prev_close']].apply(lambda x: max(x['high']-x['low'], abs(x['high']-x['prev_close']), abs(x['low']-x['prev_close'])), axis=1)
    atr = df['tr'].rolling(period).mean().iloc[-1]
    return float(atr) if not np.isnan(atr) else None

def compute_delta_from_trades(trades):
    # trades: list of trade dicts from ccxt
    # try to use 'side' if present, else approximate by comparing to mid price
    if not trades:
        return 0.0, 0.0
    buy_vol = 0.0
    sell_vol = 0.0
    for t in trades:
        size = float(t.get('amount', t.get('volume', 0) or 0))
        side = t.get('side', None)
        # try to infer side if missing
        if side is None:
            # fallback: if 'takertype' or 'maker' present
            if 'takerSide' in t:
                side = 'buy' if t['takerSide'].lower() == 'buy' else 'sell'
        if side is None:
            # as last resort: compare price with last bid/ask (we'll approximate not having orderbook here)
            # we won't do this inference here; treat as unknown
            continue
        if side.lower() == 'buy':
            buy_vol += size
        else:
            sell_vol += size
    total = buy_vol + sell_vol
    delta = buy_vol - sell_vol
    ratio = (abs(delta) / total) if total > 0 else 0.0
    # sign: positive -> net aggressive buys, negative -> net sells
    return float(delta), float(ratio)

def near_level(price, level, atr, mult=0.5):
    buffer = (atr or 0) * mult
    return abs(price - level) <= buffer

# ---------------- scoring rules ----------------
def score_reversal(symbol, ticker_price, ohlcv_tf, trades_recent, orderbook_top, params):
    """
    Возвращает dict with score, reasons, suggested SL/TP
    """
    score = 0.0
    reasons = []
    pivots = compute_pivots_from_ohlcv(ohlcv_tf)
    if not pivots:
        return {"score": 0.0, "reasons": ["no_pivot"], "side": None}
    atr = compute_atr(ohlcv_tf, period=params['atr_period']) or 0.0
    vwap = compute_vwap_from_ohlcv(ohlcv_tf)
    # compute delta
    delta, delta_ratio = compute_delta_from_trades(trades_recent)

    # 1) pivot proximity (Camarilla R3/S3 considered stronger)
    c = pivots['camarilla']
    classic = pivots['classic']
    side = None
    if near_level(ticker_price, c['R3'], atr, params['near_level_atr_mult']):
        score += 2.0
        reasons.append("near_camarilla_R3")
        side = "short"  # expect reversal down from resistance
    if near_level(ticker_price, c['S3'], atr, params['near_level_atr_mult']):
        score += 2.0
        reasons.append("near_camarilla_S3")
        side = "long"

    # 2) delta opposite (if price at resistance and buys spike -> maybe breakout, but we want opposite)
    if side == "short" and delta_ratio > params['delta_ratio_threshold'] and delta < 0:
        # if at resistance and there is net aggressive selling (delta<0) -> supports reversal down
        score += 3.0
        reasons.append("delta_confirms_sell_pressure")
    if side == "long" and delta_ratio > params['delta_ratio_threshold'] and delta > 0:
        score += 3.0
        reasons.append("delta_confirms_buy_pressure")

    # 3) vwap divergence: if price well above VWAP at resistance => more likely rejection (or vice versa)
    if vwap:
        if side == "short" and ticker_price > vwap:
            score += 1.5
            reasons.append("price_above_vwap_at_resistance")
        if side == "long" and ticker_price < vwap:
            score += 1.5
            reasons.append("price_below_vwap_at_support")

    # 4) price action rejection (last candle has long wick opposite to direction)
    # use last ohlcv candle
    last = ohlcv_tf[-2]  # last closed candle
    o_open, o_high, o_low, o_close = last[1], last[2], last[3], last[4]
    body = abs(o_close - o_open)
    upper_wick = o_high - max(o_close, o_open)
    lower_wick = min(o_close, o_open) - o_low
    if side == "short" and upper_wick > body * 1.5:
        score += 1.5
        reasons.append("upper_wick_rejection")
    if side == "long" and lower_wick > body * 1.5:
        score += 1.5
        reasons.append("lower_wick_rejection")

    # 5) liquidity depth imbalance (top N)
    try:
        bids = sum([float(x[1]) for x in orderbook_top['bids'][:5]])
        asks = sum([float(x[1]) for x in orderbook_top['asks'][:5]])
        if side == "short" and bids < asks * 0.7:
            # more asks near price -> pressure to go down
            score += 0.8
            reasons.append("ask_dominance")
        if side == "long" and asks < bids * 0.7:
            score += 0.8
            reasons.append("bid_dominance")
    except Exception:
        pass

    # produce suggested SL/TP
    if side:
        sl = None
        tp = None
        # SL: opposite side ATR multiplier
        if side == "long":
            sl = ticker_price - max(atr * 1.5, atr * 0.5)
            # TP: next pivot above or R1 classic
            tp = classic['R1']
        else:
            sl = ticker_price + max(atr * 1.5, atr * 0.5)
            tp = classic['S1']
        return {"score": round(score, 2), "reasons": reasons, "side": side, "SL": sl, "TP": tp, "atr": atr, "vwap": vwap, "delta": delta, "delta_ratio": delta_ratio, "pivots": pivots}
    else:
        return {"score": round(score, 2), "reasons": reasons, "side": None}

# ---------------- main scanner ----------------
def discover_symbols(min_volume_24h=100000):
    """Попытка получить список активных USDT perpetual через ccxt markets"""
    markets = exchange.load_markets()
    syms = []
    for symbol, md in exchange.markets.items():
        try:
            # фильтруем по quote 'USDT' либо name containing 'USDT'
            if ('USDT' in symbol) or (md.get('quote') == 'USDT'):
                # try to check if it's perpetual/swap - many markets have 'swap' in id or info
                info = md.get('info', {}) or {}
                market_type = info.get('instType') or info.get('type') or ''
                # volume:
                # beware: not all exchanges give baseVolume24h in markets; we will skip vol filter if missing
                vol24 = None
                if 'baseVolume24h' in info:
                    try:
                        vol24 = float(info.get('baseVolume24h') or 0)
                    except:
                        vol24 = None
                # Accept if swap/perp or we don't know
                if (('SWAP' in symbol.upper() or 'PERP' in symbol.upper() or 'SWAP' in market_type.upper() or 'PERPETUAL' in market_type.upper()) or True):
                    syms.append(symbol)
        except Exception:
            continue
    # dedupe and sort
    syms = sorted(list(set(syms)))
    return syms

def run_scan_loop(cfg):
    print("Scanner started at", now_ts())
    bot_token = cfg['telegram_bot_token']
    chat_id = cfg['telegram_chat_id']
    symbols = cfg['symbols'] or discover_symbols(cfg['min_base_volume_24h'])
    print(f"Symbols to scan: {len(symbols)} (example: {symbols[:10]})")
    exchange.load_markets()
    # main loop
    while True:
        try:
            t0 = time.time()
            alerts = []
            for sym in symbols:
                try:
                    # fetch ticker price
                    try:
                        ticker = exchange.fetch_ticker(sym)
                        last_price = float(ticker['last'])
                    except Exception:
                        # skip symbol if can't fetch ticker
                        continue

                    # fetch ohlcv for pivot & indicators (use 1h or 1d)
                    tf = '1h' if cfg['pivot_anchor_tf'].endswith('h') else cfg['pivot_anchor_tf']
                    ohlcv = safe_fetch_ohlcv(sym, timeframe=tf, limit=200)
                    if not ohlcv:
                        continue

                    # fetch recent trades for delta
                    try:
                        trades = exchange.fetch_trades(sym, limit=cfg['delta_window_trades'])
                    except Exception:
                        trades = []

                    # fetch orderbook top
                    try:
                        ob = exchange.fetch_order_book(sym, limit=50)
                    except Exception:
                        ob = {'bids': [], 'asks': []}

                    res = score_reversal(sym, last_price, ohlcv, trades, ob, cfg)
                    # only if score high enough and side detected
                    if res.get('side') and res.get('score', 0) >= cfg['min_confidence_score']:
                        # build message
                        msg = f"📡 <b>OKX Signal</b>\n"
                        msg += f"• <b>Пара:</b> {sym}\n"
                        msg += f"• <b>Время:</b> {now_ts()}\n"
                        msg += f"• <b>Направление:</b> {res['side'].upper()}\n"
                        msg += f"• <b>Цена входа (approx):</b> {last_price:.6f}\n"
                        if res.get('SL'):
                            msg += f"• <b>SL:</b> {res['SL']:.6f}\n"
                        if res.get('TP'):
                            msg += f"• <b>TP:</b> {res['TP']:.6f}\n"
                        msg += f"• <b>Score:</b> {res['score']} ({', '.join(res['reasons'])})\n"
                        msg += f"• <b>ATR:</b> {res.get('atr'):.6f} • <b>VWAP:</b> {res.get('vwap'):.6f}\n"
                        msg += f"• <b>Delta:</b> {res.get('delta'):.2f} • <b>DeltaRatio:</b> {res.get('delta_ratio'):.2f}\n"
                        msg += f"\n<i>Auto-scan OKX • algorithmic reversal detector</i>"
                        alerts.append((sym, msg))
                        # send immediately
                        send_telegram_message(bot_token, chat_id, msg)
                        print(f"Sent alert for {sym} score={res['score']}")

                except Exception as e_sym:
                    print("Error processing symbol", sym, e_sym)
                    traceback.print_exc()
                    continue

            t1 = time.time()
            elapsed = t1 - t0
            sleep_for = max(5, cfg['scan_interval'] - elapsed)
            print(f"[{now_ts()}] Round done. Alerts this round: {len(alerts)}. Sleeping {int(sleep_for)}s.")
            time.sleep(sleep_for)
        except KeyboardInterrupt:
            print("Scanner stopped by user.")
            break
        except Exception as e:
            print("Fatal loop error:", e)
            traceback.print_exc()
            # wait a bit before retrying
            time.sleep(10)

# ---------------- run ----------------
if __name__ == "__main__":
    # sanity checks
    if not CONFIG['telegram_bot_token'] or not CONFIG['telegram_chat_id']:
        raise SystemExit("Please set TELEGRAM token and chat_id in CONFIG before running.")
    # optional: small discovery if symbols empty
    if not CONFIG['symbols']:
        print("Discovering symbols (this may take few seconds)...")
        try:
            discovered = discover_symbols(CONFIG.get('min_base_volume_24h', 100000))
            # allow user override, but default to top 200 discovered
            CONFIG['symbols'] = discovered[:200]
            print(f"Discovered {len(CONFIG['symbols'])} symbols. Using top {len(CONFIG['symbols'])}.")
        except Exception as e:
            print("Discover error:", e)
            CONFIG['symbols'] = []

    run_scan_loop(CONFIG)

Discovering symbols (this may take few seconds)...
Discovered 200 symbols. Using top 200.
Scanner started at 2025-10-08 23:28:11 UTC
Symbols to scan: 200 (example: ['0G/USDT:USDT', '1INCH/USDT', '1INCH/USDT:USDT', '2Z/USDT:USDT', 'A/USDT', 'A/USDT:USDT', 'AAVE/USDT', 'AAVE/USDT:USDT', 'ACA/USDT', 'ACE/USDT'])


/tmp/ipython-input-2627281072.py:57: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
/tmp/ipython-input-2627281072.py:57: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


Sent alert for 0G/USDT:USDT score=7.0


/tmp/ipython-input-2627281072.py:57: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


Sent alert for 1INCH/USDT score=6.3


...

Sent alert for DAO/USDT score=6.3


/tmp/ipython-input-2627281072.py:57: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


Sent alert for DOGE/USDT score=7.8


/tmp/ipython-input-2627281072.py:57: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


Sent alert for DUCK/USDT score=6.3
